In [1]:
from sqlalchemy import create_engine

In [2]:
connection_string = "{drivername}+{dbapi}://{user}:{passwd}@{host}:{port}/{db_name}?charset=utf8".format(
     drivername="mysql",
     dbapi="mysqldb",
     user="root",
     passwd="my-secret-pw",
     host="localhost",
     port="3306",
     db_name="examples",
)
engine = create_engine(connection_string, echo=True, future=True)

In [3]:
from sqlalchemy import Table, Column, ForeignKey, Integer, String, Text, DateTime

from sqlalchemy.orm import declarative_base
from sqlalchemy.orm import relationship

In [4]:
Base = declarative_base()

In [5]:
quote_tag = Table(
    'quote_tag',
    Base.metadata,
    Column('quote_id', ForeignKey('quote.id'), primary_key=True),
    Column('tag_id', ForeignKey('tag.id'), primary_key=True)
)

In [6]:
class Tag(Base):
    __tablename__ = "tag"

    id = Column(Integer, primary_key=True)
    name = Column('name', String(100), unique=True)
    quotes = relationship('Quote', secondary=quote_tag, back_populates="tags")  # M-to-M for quote and tag
    
    def __repr__(self):
        return f"Tag(id={self.id!r}, name={self.name!r})"

In [7]:
class Quote(Base):
    __tablename__ = "quote"

    id = Column(Integer, primary_key=True)
    text = Column('text', Text())
    author_id = Column(Integer, ForeignKey('author.id'))  # Many quotes to one author
    author = relationship("Author", back_populates="quotes")
    tags = relationship("Tag", secondary=quote_tag, back_populates="quotes")
    
    def __repr__(self):
        return f"Quote(id={self.id!r}, text={self.text!r})"

In [8]:
class Author(Base):
    __tablename__ = "author"
    
    id = Column(Integer, primary_key=True)
    name = Column('name', String(30), unique=True)
    birthday = Column('birthday', DateTime)
    bornlocation = Column('bornlocation', String(150))
    description = Column('description', Text())
    quotes = relationship("Quote", back_populates="author", cascade="all, delete-orphan")
    
    def __repr__(self):
        return f"Author(id={self.id!r}, name={self.name!r}, birthday={self.birthday!r})"

In [9]:
 Base.metadata.create_all(engine)

2022-05-17 18:52:56,199 INFO sqlalchemy.engine.Engine SELECT DATABASE()
2022-05-17 18:52:56,201 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-05-17 18:52:56,203 INFO sqlalchemy.engine.Engine SELECT @@sql_mode
2022-05-17 18:52:56,204 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-05-17 18:52:56,205 INFO sqlalchemy.engine.Engine SELECT @@lower_case_table_names
2022-05-17 18:52:56,206 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-05-17 18:52:56,208 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-05-17 18:52:56,209 INFO sqlalchemy.engine.Engine SELECT COUNT(*) FROM information_schema.tables WHERE table_schema = %s AND table_name = %s
2022-05-17 18:52:56,210 INFO sqlalchemy.engine.Engine [generated in 0.00074s] ('examples', 'quote_tag')
2022-05-17 18:52:56,213 INFO sqlalchemy.engine.Engine SELECT COUNT(*) FROM information_schema.tables WHERE table_schema = %s AND table_name = %s
2022-05-17 18:52:56,214 INFO sqlalchemy.engine.Engine [cached since 0.004132s ago] ('examples', 'tag')
2

In [10]:
from sqlalchemy.orm import Session

In [11]:
session = Session(engine)

In [12]:
import dateutil.parser as parser
from dateutil.parser import ParserError

In [13]:
author = Author(
    name='Albert Einstein', 
    birthday=parser.parse('March 14, 1879'), 
    bornlocation='Ulm, Germany',
    description="""
        In 1879, Albert Einstein was born in Ulm, Germany. He completed his Ph.D. at the University of Zurich by 1909. His 1905 paper explaining the photoelectric effect, the basis of electronics, earned him the Nobel Prize in 1921. His first paper on Special Relativity Theory, also published in 1905, changed the world. After the rise of the Nazi party, Einstein made Princeton his permanent home, becoming a U.S. citizen in 1940. Einstein, a pacifist during World War I, stayed a firm proponent of social justice and responsibility. He chaired the Emergency Committee of Atomic Scientists, which organized to alert the public to the dangers of atomic warfare.At a symposium, he advised: "In their struggle for the ethical good, teachers of religion must have the stature to give up the doctrine of a personal God, that is, give up that source of fear and hope which in the past placed such vast power in the hands of priests. In their labors they will have to avail themselves of those forces which are capable of cultivating the Good, the True, and the Beautiful in humanity itself. This is, to be sure a more difficult but an incomparably more worthy task . . . " ("Science, Philosophy and Religion, A Symposium," published by the Conference on Science, Philosophy and Religion in their Relation to the Democratic Way of Life, Inc., New York, 1941). In a letter to philosopher Eric Gutkind, dated Jan. 3, 1954, Einstein stated: "The word god is for me nothing more than the expression and product of human weaknesses, the Bible a collection of honorable, but still primitive legends which are nevertheless pretty childish. No interpretation no matter how subtle can (for me) change this," (The Guardian, "Childish superstition: Einstein's letter makes view of religion relatively clear," by James Randerson, May 13, 2008). D. 1955.While best known for his mass–energy equivalence formula E = mc2 (which has been dubbed "the world's most famous equation"), he received the 1921 Nobel Prize in Physics "for his services to theoretical physics, and especially for his discovery of the law of the photoelectric effect". The latter was pivotal in establishing quantum theory.Einstein thought that Newtonion mechanics was no longer enough to reconcile the laws of classical mechanics with the laws of the electromagnetic field. This led to the development of his special theory of relativity. He realized, however, that the principle of relativity could also be extended to gravitational fields, and with his subsequent theory of gravitation in 1916, he published a paper on the general theory of relativity. He continued to deal with problems of statistical mechanics and quantum theory, which led to his explanations of particle theory and the motion of molecules. He also investigated the thermal properties of light which laid the foundation of the photon theory of light.He was visiting the United States when Adolf Hitler came to power in 1933 and did not go back to Germany. On the eve of World War II, he endorsed a letter to President Franklin D. Roosevelt alerting him to the potential development of "extremely powerful bombs of a new type" and recommending that the U.S. begin similar research. This eventually led to what would become the Manhattan Project. Einstein supported defending the Allied forces, but largely denounced the idea of using the newly discovered nuclear fission as a weapon. Later, with Bertrand Russell, Einstein signed the Russell–Einstein Manifesto, which highlighted the danger of nuclear weapons. Einstein was affiliated with the Institute for Advanced Study in Princeton, New Jersey, until his death in 1955.His great intellectual achievements and originality have made the word "Einstein" synonymous with genius.More: http://en.wikipedia.org/wiki/Albert_E...http://www.nobelprize.org/nobel_prize...    
    """
)

In [14]:
quote=Quote(text="The world as we have created it is a process of our thinking. It cannot be changed without changing our thinking.", author=author)

In [15]:
tag_change=Tag(name="change")
tag_deep_thought=Tag(name="deep-thoughts")
tag_thinking=Tag(name="thinking")
tag_world=Tag(name="world")

In [16]:
quote.tags.append(tag_change)
quote.tags.append(tag_deep_thought)
quote.tags.append(tag_thinking)
quote.tags.append(tag_world)

In [17]:
quote

Quote(id=None, text='The world as we have created it is a process of our thinking. It cannot be changed without changing our thinking.')

In [18]:
quote.tags

[Tag(id=None, name='change'),
 Tag(id=None, name='deep-thoughts'),
 Tag(id=None, name='thinking'),
 Tag(id=None, name='world')]

In [19]:
tag_deep_thought.quotes

[Quote(id=None, text='The world as we have created it is a process of our thinking. It cannot be changed without changing our thinking.')]

In [20]:
session.add(quote)

In [21]:
session.commit()

2022-05-17 18:52:56,476 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-05-17 18:52:56,477 INFO sqlalchemy.engine.Engine INSERT INTO author (name, birthday, bornlocation, description) VALUES (%s, %s, %s, %s)
2022-05-17 18:52:56,478 INFO sqlalchemy.engine.Engine [generated in 0.00078s] ('Albert Einstein', datetime.datetime(1879, 3, 14, 0, 0), 'Ulm, Germany', '\n        In 1879, Albert Einstein was born in Ulm, Germany. He completed his Ph.D. at the University of Zurich by 1909. His 1905 paper explaining th ... (3554 characters truncated) ... have made the word "Einstein" synonymous with genius.More: http://en.wikipedia.org/wiki/Albert_E...http://www.nobelprize.org/nobel_prize...    \n    ')
2022-05-17 18:52:56,482 INFO sqlalchemy.engine.Engine INSERT INTO tag (name) VALUES (%s)
2022-05-17 18:52:56,482 INFO sqlalchemy.engine.Engine [generated in 0.00069s] ('change',)
2022-05-17 18:52:56,485 INFO sqlalchemy.engine.Engine INSERT INTO tag (name) VALUES (%s)
2022-05-17 18:52:56,486 INFO 